<a href="https://colab.research.google.com/github/lakhanrajpatlolla/aiml-learning/blob/master/Lakhan_Unit4_MH2_RAG_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#Importing packages
!pip -q install openai
!pip -q install langchain
!pip -q install langchain-openai
!pip -q install pypdf
!pip -q install chromadb
!pip -q install tiktoken
!pip -q install langchain_community

import os
import openai
import numpy as np

In [19]:
#Authentication for OpenAI API
f = open('/content/IIITH_Hackathon_RAG.txt')
#f = open('/content/key-2.txt')

api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [20]:
# Step 1: Load the document files (1 point)

from langchain_community.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose
    PyPDFLoader("/content/AST-1.pdf"),
    PyPDFLoader("/content/AST-2.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

print(docs[0].page_content)

Module 3: AST-1  
 
TITLE: Transitioning from Research to Production Environment 
LEARNING OBJECTIVES: 
At the end of the experiment, you will be able to transition from the research environment to the 
production environment. You will understand the concept of modularization and convert the model 
developed in Jupyter Notebook into different modules tailored to specific functionalities: Data 
Manager, Training, Pipeline, Predict, etc. 
 
You will be able to understand and implement the following aspects: 
 
1. VS code setup and understanding interface components  
2. Creating virtual environment, installing requirement, running .py & .ipynb file 
3. Understanding __name__ == “__main__” and building our own modules 
4. Modularization: Converting a ML model from research environment format (Notebook) 
to product environment format (.py files) 
A) Creating different modules specific to functionality: Data Manager, Training, 
Pipeline, Predict 
B) Understanding folder structure 
C) Separa

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [22]:
# Step 2: Split the loaded documents into chunks (1 point)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

splits = text_splitter.split_documents(docs)
print(len(splits))
#splits

40


In [24]:
#Step 3: Create embeddings and store in Vector Database (3 points)

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma # Light-weight and in memory

embedding = OpenAIEmbeddings()

# Change to your current directory
persist_directory = './chroma/'
!rm -rf ./chroma  # remove old database files if any


vectordb = Chroma.from_documents(
    documents=splits, # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory # save the directory
)

vectordb.persist()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# Similarity search without MMR

question = "how does software developement works?"

docs = vectordb.similarity_search(question,k=2) # k --> No. of doc as return
print(len(docs))
print(docs[0].page_content)
print(docs[1].page_content)


In [ ]:
# Addressing Diversity - MMR-Maximum Marginal Relevance
docs_with_mmr=vectordb.max_marginal_relevance_search(question, k=2, fetch_k=6) # With MMR

docs_with_mmr[0]
docs_with_mmr[1]


In [ ]:
#Step 4: Perform the retrieval augmented generation by integrating with LLM (3 points)

#Without MMR

from langchain.chains import RetrievalQA

llm_name = "gpt-3.5-turbo"
print(llm_name)

question = "What is software development cycle ?"
docs = vectordb.max_marginal_relevance_search(question, k=2, fetch_k=3)
len(docs)

#vetore store backed retriever
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
retriever = vectordb.as_retriever()

qa_chain = RetrievalQA.from_chain_type(llm, retriever= retriever, return_source_documents=True)

result = qa_chain.invoke({"query": question})

result["result"]
result["source_documents"]

In [ ]:
# Step 5: Frame at least five logical questions relevant to the knowledge base and demonstrate relevant answers from the RAG system (1 point)


questions = [
    "What are the benefits of modularization in software development?",
    "How does modularization impact software testing?",
    "What are the key functionalities of a Data Manager module?",
    "Explain the role of the Training module in the software development process.",
    "What are the different types of software testing methodologies?",
]

# Get answers for each question
for question in questions:
    result = qa_chain.invoke({"query": question})
    print(f"Question: {question}")
    print(f"Answer: {result['result']}\n")
    # Optionally print source documents:
    # print(f"Source Documents: {result['source_documents']}\n")

In [ ]:
!pip install langchainhub

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain with MMR
# Run chain
from langchain.chains import RetrievalQA
qa_chain_mmr = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 2, "fetch_k":6} ), # "k":2, "fetch_k":3
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True
                                       )
# Get answers for each question
for question in questions:
    result = qa_chain_mmr.invoke({"query": question})
    print(f"Question: {question}")
    print(f"Answer: {result['result']}\n")
    # Optionally print source documents:
    # print(f"Source Documents: {result['source_documents']}\n")


In [ ]:
#Step 6: Create a Gradio App  where user can write the query and get the response from the RAG system(1 point)

!pip install gradio

import gradio as gr
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI


def get_answer(query):
    """Gets the answer from the RAG system."""
    result = qa_chain.invoke({"query": query})
    return result["result"]

# Create the Gradio interface
iface = gr.Interface(
    fn=get_answer,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Textbox(lines=5),
    title="Software Development RAG System",
    description="Ask questions about software development best practices.",
)

# Launch the app
iface.launch()